Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/alvaro/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # this layer corresponds to a stack of filters which will be convolved with
  # the input image, creating a new image with dimensions 
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  # the new image is convolved with a new stack of filters. 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
  # we end with 2 layers of fully connected layers
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # convolve with stride 2, same padding, output size (mini_batch_size)x14x14x16
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # again we have stride 2, same padding, output size (mini_batch_size)x7x7x16
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    # reshape, so we have now shape (mini_batch_size)x(7*7*16)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    # 1 fully connected hidden layer + another fully connected, output layer
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.165815
Minibatch accuracy: 12.5%
Validation accuracy: 9.3%
Minibatch loss at step 50: 1.235685
Minibatch accuracy: 62.5%
Validation accuracy: 58.6%
Minibatch loss at step 100: 1.078500
Minibatch accuracy: 68.8%
Validation accuracy: 72.9%
Minibatch loss at step 150: 1.305781
Minibatch accuracy: 62.5%
Validation accuracy: 73.0%
Minibatch loss at step 200: 0.561904
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 250: 1.868404
Minibatch accuracy: 56.2%
Validation accuracy: 71.3%
Minibatch loss at step 300: 0.920869
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 350: 1.460248
Minibatch accuracy: 62.5%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.443218
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.299284
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.810335
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [116]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # this layer corresponds to a stack of filters which will be convolved with
  # the input image, creating a new image with dimensions 
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  # the new image is convolved with a new stack of filters. 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
  # we end with 2 layers of fully connected layers
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # convolve with stride 2, same padding, output size (mini_batch_size)x14x14x16
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # again we have stride 2, same padding, output size (mini_batch_size)x7x7x16
    conv = tf.nn.conv2d(pooled, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    shape = pooled.get_shape().as_list()
    # reshape, so we have now shape (mini_batch_size)x(7*7*16)
    reshape = tf.reshape(pooled, [shape[0], shape[1] * shape[2] * shape[3]])
    # 1 fully connected hidden layer + another fully connected, output layer
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [117]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.253628
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.305754
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 2.301203
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 150: 2.299738
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 200: 2.305777
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 250: 2.302049
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 300: 2.311211
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 350: 2.296919
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 400: 2.311899
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 450: 2.296807
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 500: 2.312230
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibat

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [170]:
def dropout_relu(*args, **kwargs):
    return tf.nn.dropout(*args, **kwargs, keep_prob=0.6)

In [173]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # this layer corresponds to a stack of filters which will be convolved with
  # the input image, creating a new image with dimensions 
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  # the new image is convolved with a new stack of filters. 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
  # we end with 2 layers of fully connected layers
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, train=False):
    if train:
        relu = dropout_relu
    else:
        relu = tf.nn.relu
    # convolve with stride 2, same padding, output size (mini_batch_size)x14x14x16
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = relu(conv + layer1_biases)
    pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # again we have stride 2, same padding, output size (mini_batch_size)x7x7x16
    conv = tf.nn.conv2d(pooled, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = relu(conv + layer2_biases)
    pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    shape = pooled.get_shape().as_list()
    # reshape, so we have now shape (mini_batch_size)x(7*7*16)
    reshape = tf.reshape(pooled, [shape[0], shape[1] * shape[2] * shape[3]])
    # 1 fully connected hidden layer + another fully connected, output layer
    hidden = relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, train=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [174]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('\nMinibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 8.822244
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%

Minibatch loss at step 100: 0.835718
Minibatch accuracy: 75.0%
Validation accuracy: 62.7%

Minibatch loss at step 200: 0.926237
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%

Minibatch loss at step 300: 0.635276
Minibatch accuracy: 81.2%
Validation accuracy: 73.8%

Minibatch loss at step 400: 0.698110
Minibatch accuracy: 79.7%
Validation accuracy: 78.5%

Minibatch loss at step 500: 0.906767
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%

Minibatch loss at step 600: 0.513691
Minibatch accuracy: 82.8%
Validation accuracy: 79.0%

Minibatch loss at step 700: 0.878307
Minibatch accuracy: 76.6%
Validation accuracy: 78.8%

Minibatch loss at step 800: 0.798647
Minibatch accuracy: 76.6%
Validation accuracy: 81.5%

Minibatch loss at step 900: 0.703893
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%

Minibatch loss at step 1000: 0.702931
Minibatch accuracy: 82.8%
Validation accu